<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Neural networks </h2>	


Build a neural network classifier using an architecture of your choosing. This application
of deep learning can be done in PyTorch, TensorFlow, or a framework of your choice. This is the
industry standard for image classification. Describe your network and assess its performance. To
receive extra credit, your neural network classifier must outperform your other methods.



In [41]:
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from PIL import Image, ImageOps

In [42]:
def NN():
    classifier = Sequential()

    classifier.add(Conv2D(32, (3, 3), input_shape = (256, 256, 3), activation = 'relu', padding="same"))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    classifier.add(Flatten())

    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 20, activation = 'softmax'))
    
    return classifier

In [43]:
classifier = NN()
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [44]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('nn_train',
                                    target_size = (256, 256),
                                    batch_size = 32,
                                    class_mode = 'categorical')

Found 1202 images belonging to 20 classes.


In [45]:
val_set = val_datagen.flow_from_directory('nn_val',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 270 images belonging to 20 classes.


In [46]:
filepath = "saved-model-{epoch:02d}-new.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [47]:
classifier.fit_generator(training_set,
                            steps_per_epoch = 2000,
                            epochs = 5,
                            validation_data = val_set,
                            validation_steps = 500)

Epoch 1/5
   2/2000 [..............................] - ETA: 1:29:31 - loss: 21.2460 - accuracy: 0.0312

KeyboardInterrupt: 

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('nn_test',
                                            target_size = (256, 256),
                                            shuffle = False,
                                            batch_size = 1,
                                            class_mode = None,)

In [ ]:
test_set.reset()
filenames = test_set.filenames

predict = classifier.predict_generator(test_set, steps=len(test_set))

In [ ]:
predicted_class_indices=np.argmax(predict,axis=1)

In [ ]:
df = pd.DataFrame(predicted_class_indices)
df.to_csv('nn_prediction.csv')

In [ ]:
classifier = load_model('saved-model-01.hdf5')